Run 'setup.py' file before this one using instruction in the beginning

In [267]:
import sys
print(sys.executable)

/home/theorcos239/structured-randomized-decompositions/venv/bin/python


In [268]:
import time
from scipy.linalg import qr # type: ignore
from scipy.linalg import svd # type: ignore
import numpy as np # type: ignore
import transforms # type: ignore

def randomized_svd(A, k, p, f_matmul):

  start = time.time()

  S = np.random.randn(n, k + p)
  Y = f_matmul(A, S)

  print("matmul in rand :", time.time() - start)

  start = time.time()

  Q, _ = qr(Y, mode='economic')

  B = f_matmul(Q.T, A)


  U, s, V = svd(B, full_matrices=False)

  U = f_matmul(Q, U)

  print("qr in rand :", time.time() - start)

  return U, s, V



In [269]:
n = 2**10
m = 2**10
k = 100
p = 10
A = np.zeros((n, m))
for i in range(0, n):
  A[i][i] = 2
  if i > 0:
    A[i - 1][i] = -1
  if i < n - 1:
    A[i + 1][i] = -1


#s_rand = randomized_svd(A, k, p)
#print(s_rand)

In [270]:
def my_matmul(A, B):
  n, m = A.shape
  _, k = B.shape
  C = np.zeros((n, k))
  for i in range(0, n):
    for j in range(0, k):
      for l in range(0, m):
        C[i][j] += A[i][l] * B[l][j]
  return C

In [271]:
def matmul(A, B):
  return A @ B

In [272]:
import numpy as np # type: ignore
from scipy.linalg import svd # type: ignore

def fast_hadamard_transform(x):
    n = len(x)
    h = 1
    while h < n:
        for i in range(0, n, h * 2):
            for j in range(i, min(i + h, n - h)):
                  x_j = x[j]
                  x[j] = x_j + x[j + h]
                  x[j + h] = x_j - x[j + h]
        h *= 2
    return x / np.sqrt(n)

def hadamard_svd(A, k, f_matmul, p=5, random = False):
    m, n = A.shape
    power = 1
    l = k + p
    start = time.time()
    if random:
        Omega = np.random.randn(n, l)
        for i in range(0, n):
          Omega[i, :] = transforms.fast_hadamard_transform(Omega[i, :])
        Y = f_matmul(A, Omega)
    else:
        Y = A
        for i in range(0, m):
          Y[i, :] = transforms.fast_hadamard_transform(Y[i, :])
        random_cols = np.random.choice(n, l, replace = False)
        Y = Y[:, random_cols]


    start = time.time()

    Q, _ = qr(Y, mode='economic')

    B = f_matmul(Q.T, A)

    U, s, Vt = svd(B, full_matrices=False)

    U = f_matmul(Q, U)

    return U, s, Vt[:, :n]


In [273]:
def format_from_svd(U, s, V):
  n, m = U.shape
  full = np.zeros(m)
  full[:len(s)] = s
  S = np.diag(full)
  return U @ S @ V

In [274]:
import time

In [275]:
#A = np.random.randn(n, m)
v = np.random.randn(n, k+p)
w = np.random.randn(k+p, m)
A = v @ w

import time
import numpy as np # type: ignore
from scipy.linalg import svd # type: ignore

start = time.time()
U_h_r, s_hadamar_r, V_h_r = hadamard_svd(A, k, matmul, p, random = False)
print(time.time() - start)

start = time.time()
U_h, s_hadamar, V_h = hadamard_svd(A, k, matmul, p, random = True)
print(time.time() - start)

start = time.time()
U_r, s_rand, V_r = randomized_svd(A, k, p, matmul)
print(time.time() - start)

# start = time.time()
# U, s, V = np.linalg.svd(A)
# print(time.time() - start)

# start = time.time()
# U_h_r, s_hadamar_r, V_h_r = hadamard_svd(A, k, my_matmul, p, random = False)
# print(time.time() - start)

# start = time.time()
# U_r, s_rand, V_r = randomized_svd(A, k, p, my_matmul)
# print(time.time() - start)

1.1069245338439941
1.3975772857666016
matmul in rand : 0.050304412841796875
qr in rand : 1.2197926044464111
1.2709403038024902


In [276]:
#print(np.linalg.norm(format_from_svd(U, s, V) - A))
print(np.linalg.norm(format_from_svd(U_h, s_hadamar, V_h) - A))
print(np.linalg.norm(format_from_svd(U_h_r, s_hadamar_r, V_h_r) - A))
print(np.linalg.norm(format_from_svd(U_r, s_rand, V_r) - A))

8.059096190112214e-11
1.324754542557833e-10
1.8362883917779976e-10


In [277]:
def fft(x):
  n = len(x)
  if n == 1:
      return x
  even = fft(x[::2])
  odd = fft(x[1::2])
  rotate = np.exp(-2 * np.pi * np.arange(n // 2) / n)
  first_half = even + rotate * odd
  second_half = even - rotate * odd
  return np.concatenate([first_half, second_half])

In [278]:
def fft_svd(A, k, f_matmul, p=5):
  m, n = A.shape
  Y = A
  l = k + p

  start = time.time()
  for i in range(0, m):
    Y[i, :] = transforms.fft(Y[i, :])

  random_cols = np.random.choice(n, l, replace = False)
  Y = Y[:, random_cols]
  print("fft calc:", time.time() - start)
  start = time.time()

  Q, _ = qr(Y, mode='economic')

  B = f_matmul(Q.T, A)

  U, s, Vt = svd(B, full_matrices=False)

  U = f_matmul(Q, U)

  print("qr in fft :", time.time() - start)

  return U, s, Vt[:, :n]



In [279]:
start = time.time()
U_fft, s_fft, V_fft = fft_svd(A, k, matmul, p)
print(time.time() - start)
print(np.linalg.norm(format_from_svd(U_fft, s_fft, V_fft) - A))

fft calc: 0.11398577690124512
qr in fft : 0.4505312442779541
0.5654604434967041
5.092244786971548e-10


In [280]:
def struct_matrix_with_sing(sing : np.array, n: int, m: int):
    rank = len(sing)
    Rand1 = np.random.randn(n, rank)
    U, _ = np.linalg.qr(Rand1)
    Rand2 = np.random.randn(m, rank)
    V, _ = np.linalg.qr(Rand2)
    return format_from_svd(U, sing, V.T)

In [281]:
n = 512
m = 512
rank = 100 

In [282]:
from tqdm import tqdm

In [283]:
epochs = 500
result = []
bound = 50
largest = 1000

for i in tqdm(range(1, epochs)):
    s = [largest] * (bound + 10)
    s[bound : bound + 10] = [i] * 10
    s[bound] = i
    s = np.array(s)
    X = struct_matrix_with_sing(s, n, m)
    U_h_r, s_hadamar_r, V_h_r = hadamard_svd(X, bound , matmul, 0)
    res = np.linalg.norm(format_from_svd(U_h_r, s_hadamar_r, V_h_r) - X)
    result.append(res)


100%|██████████| 499/499 [04:19<00:00,  1.93it/s]


In [285]:
import plotly.express as plt
print(max(result))

plt.line(x = list(range(1, epochs)), y = result)

2762.047983686438
